### Multi-input Network - Pytorch

Prediction action sequence that connects two observations

###### structure
- look into general structure of the NeurIPS project / Martius Project

###### waypoint executer
- write program that finds path between waypoints

###### saving
- create hash and csv file that collects all the different saved networks (and their associated parameter settings)

###### data
- build multiple action prediction dataset (for n steps inbetween)
- check image normalization (with transform!)

###### training
- write dict to pass to train function with parameters (+ possibility to have addititonal outputs)

###### testing (most important)
- Multiple action prediction (How to shuffle data for long seq.?) /  create dataset with n steps appart
- Siam network: https://colab.research.google.com/drive/1sn7BDKVvi8-Ng37gvfyNw8OCf8kZY91o?usp=sharing

In [2]:
'''
Imports external and own libraries
'''

import pickle

import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader

from prettytable import PrettyTable

# own
import collector
import action
import world
import plot
import preprocess
import nets
import train

In [3]:
'''
Load data with pickle (deserialize)
'''
with open('datasets/oracle_data.pickle', 'rb') as handle:
    oracle_data = pickle.load(handle)

with open('datasets/oracle_reversed_data.pickle', 'rb') as handle:
    oracle_reversed_data = pickle.load(handle)

with open('datasets/oracle_random_data.pickle', 'rb') as handle:
    oracle_random_data = pickle.load(handle)

with open('datasets/oracle_reversed_random_data.pickle', 'rb') as handle:
    oracle_reversed_random_data = pickle.load(handle)
    
with open('datasets/random_data.pickle', 'rb') as handle:
    random_data = pickle.load(handle)

In [4]:
'''
Use Dataloader to make the data ready for the training loop
'''

train_data, test_data = preprocess.split_data(oracle_reversed_data, 0.8)

# preprocess trainingset 
oracle_train_data = preprocess.ObtainDataset(train_data, 'observations', 'actions')
oracle_test_data = preprocess.ObtainDataset(test_data, 'observations', 'actions')

# build dataloader (tensor format)
batch_size = 64
dataset_loader_train_data = DataLoader(oracle_train_data, batch_size=batch_size, shuffle=True)
dataset_loader_test_data = DataLoader(oracle_test_data, batch_size=batch_size, shuffle=True)

In [10]:
'''
Initialize dual input model
'''

DualInput_model = nets.DualInput()
summary(DualInput_model, [(3,32,32), (3,32,32)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             456
         MaxPool2d-2            [-1, 6, 14, 14]               0
            Conv2d-3           [-1, 16, 10, 10]           2,416
         MaxPool2d-4             [-1, 16, 5, 5]               0
            Conv2d-5            [-1, 6, 28, 28]             456
         MaxPool2d-6            [-1, 6, 14, 14]               0
            Conv2d-7           [-1, 16, 10, 10]           2,416
         MaxPool2d-8             [-1, 16, 5, 5]               0
            Linear-9                  [-1, 120]          48,120
           Linear-10                   [-1, 84]          10,164
           Linear-11                    [-1, 4]             340
Total params: 64,368
Trainable params: 64,368
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 36.00
Forward/b

In [6]:
'''
Make use of TensorBoard for visualizing logged results
'''

%load_ext tensorboard
%tensorboard --logdir=runs

In [13]:
'''
Train the network
'''

criterion = nn.CrossEntropyLoss()
#optimizer = optim.SGD(DualInput_model.parameters(), lr=0.01, momentum=0.9) # for small one 0.01 works well
optimizer = optim.RMSprop(DualInput_model.parameters(), lr=0.01)

# Print model's state_dict
print("Model's state_dict:")
for param_tensor in DualInput_model.state_dict():
    print(param_tensor, "\t", DualInput_model.state_dict()[param_tensor].size())

# Print optimizer's state_dict
print("\nOptimizer's state_dict:")
for var_name in optimizer.state_dict():
    print(var_name, "\t", optimizer.state_dict()[var_name])

episodes = 10
DualInput_model, test_loss, train_loss, test_acc, train_acc  = train.DualInput(dataset_loader_train_data, dataset_loader_test_data, DualInput_model, criterion, optimizer, episodes)


Model's state_dict:
conv1.weight 	 torch.Size([6, 3, 5, 5])
conv1.bias 	 torch.Size([6])
conv2.weight 	 torch.Size([16, 6, 5, 5])
conv2.bias 	 torch.Size([16])
fc1.weight 	 torch.Size([120, 400])
fc1.bias 	 torch.Size([120])
fc2.weight 	 torch.Size([84, 120])
fc2.bias 	 torch.Size([84])
fc3.weight 	 torch.Size([4, 84])
fc3.bias 	 torch.Size([4])

Optimizer's state_dict:
state 	 {}
param_groups 	 [{'lr': 0.01, 'momentum': 0, 'alpha': 0.99, 'eps': 1e-08, 'centered': False, 'weight_decay': 0, 'params': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]}]


Progress:   0%|          | 0/10 [00:00<?, ? Episode/s]

Epoch: 1, Train Loss: 1.148, Train Acc: 41 %,  Test Loss: 1.194, Test Acc: 37 %,


KeyboardInterrupt: 

In [ ]:
'''
Plot loss and accuracy curves for training and test set
'''

plot.plot_losses(test_loss, train_loss)
plot.plot_acc(test_acc, train_acc, smooth=True)

In [ ]:
'''
Show example classifications and plot confusion matrix
'''

plot.show_example_classificataions(dataset_loader_train_data, DualInput_model, amount=8)
plot.plot_confusion_matrix(dataset_loader_train_data, model)

In [ ]:
# write a function that builds a dataset with x, x+n, and the n-actions inbetween

In [ ]:
'''
Preprocess images (Normalize inputs)
'''

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

dataloader = torch.utils.data.DataLoader(oracle_train_data, batch_size=4096, shuffle=False, num_workers=4)

pop_mean = []
pop_std0 = []
pop_std1 = []
for i, data in enumerate(dataloader, 0):
    # shape (batch_size, 3, height, width)
    numpy_image = data['image'].numpy()
    
    # shape (3,)
    batch_mean = np.mean(numpy_image, axis=(0,2,3))
    batch_std0 = np.std(numpy_image, axis=(0,2,3))
    batch_std1 = np.std(numpy_image, axis=(0,2,3), ddof=1)
    
    pop_mean.append(batch_mean)
    pop_std0.append(batch_std0)
    pop_std1.append(batch_std1)

# shape (num_iterations, 3) -> (mean across 0th axis) -> shape (3,)
pop_mean = np.array(pop_mean).mean(axis=0)
pop_std0 = np.array(pop_std0).mean(axis=0)
pop_std1 = np.array(pop_std1).mean(axis=0)

In [ ]:
#for name, param in net.named_parameters():
#    print(name, param.grad.abs().sum())
#print('------------------')